In [ ]:
import sys
from pathlib import Path
import json
import pandas as pd
from src.utils import DATA_PROCESSED
pd.set_option("display.max_colwidth", None)

In [ ]:
ROOT = Path.cwd().resolve()
if (ROOT / "src" / "sentiment").exists():
    pass
elif (ROOT.parent / "src" / "sentiment").exists():
    ROOT = ROOT.parent
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))

In [ ]:
processed_files = sorted(DATA_PROCESSED.glob("processed_*.jsonl"), key=lambda p: p.stat().st_mtime, reverse=True)
if not processed_files:
    raise FileNotFoundError("No data/processed/processed_*.jsonl found. Run: python scripts/run_process.py")
processed_path = processed_files[0]
print(f"Loaded: {processed_path.name}")
rows = []
with open(processed_path, encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        try:
            rows.append(json.loads(line))
        except json.JSONDecodeError:
            continue
df = pd.DataFrame(rows)

In [ ]:
# Inspect shape and sample
df.head()

In [ ]:
sent_cols = [c for c in df.columns if c.startswith("sentiment_")]
print(df[sent_cols].describe())
if len(sent_cols) >= 2:
    print("\nCorrelation:")
    print(df[sent_cols].corr())


## Testing is-ai-related flag

In [ ]:
cols = ["headline", "is_ai_related","url"]
headline_flags = df[cols].drop_duplicates(subset=["headline"]).reset_index(drop=True)

headline_flags.loc[headline_flags["is_ai_related"] == False]

In [ ]:
headline_flags.loc[headline_flags["is_ai_related"] == True]